# BPE Token Sequence List를 활용한 LSTM+CNN 기반 악성코드 탐지 모델
- accuracy: 0.9912482726853984
- BPE Token을 생성하면서 같이 제작된 BPE Token list를 데이터로 사용했습니다.
- 해당 모델 구조를 학습시키 위해서는 pefile dataset(dikedataset을 추천합니다.)을 utils.py의 함수들을 활용하여 전처리를 한 뒤 사용하셔야 합니다.
- 전처리를 하는 방식에 대해서는 utils 디렉토리에 있는 test.ipynb 파일을 통해 확인하실 수 있습니다.

# 필요 모듈 설치

In [ ]:
!pip install pefile
!pip install capstone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 505 kB/s 
  Created wheel for pefile: filename=pefile-2021.9.3-py3-none-any.whl size=68844 sha256=fa6743cc6282480e61bf188708b7551a0dc47118964cde6deb7f1541a27d2723
  Stored in directory: /root/.cache/pip/wheels/50/be/9b/d19a6151b9c6e303f823faedf03193d7e90dac4009a0bdafeb
Successfully built pefile
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 12.2 MB/s 


# utils.py 파일이 있는 디렉토리로 위치 이동

In [ ]:
%cd /content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test

/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test


# 필요 패키지 로드

In [ ]:
from utils import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.layers import Conv1D, MaxPooling1D, Dropout, Activation
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing import sequence
from keras.utils import np_utils

from keras.utils.np_utils import to_categorical

import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import time
import tqdm
from tqdm import tqdm_notebook
tqdm_notebook()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


0it [00:00, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data load

## OpcodeSequenceList load

In [ ]:
with open('/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/files/BPE Database/2000opcodeList_checkpoint.p', 'rb') as file:
    opcodeSequenceList = pickle.load(file)

print(opcodeSequenceList[0])

[['push', 'mov', 'sub'], ['push', 'push', 'push'], ['mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'call'], ['mov', 'cmp', 'jbe'], ['mov', 'mov', 'mov', 'mov', 'mov', 'cmp'], ['jmp'], ['push', 'call'], ['mov', 'push', 'mov', 'push', 'call', 'add'], ['mov', 'adc'], ['xchg', 'cmp'], ['jo'], ['inc', 'add'], ['push', 'push', 'call'], ['xor', 'pop', 'pop', 'pop', 'mov'], ['pop', 'ret', 'push', 'mov', 'sub'], ['push', 'push', 'push'], ['mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov'], ['mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov', 'mov'], ['mov', 'mov', 'cmp', 'je'], ['mov', 'mov', 'mov'], ['push', 'mov'], ['shl'], ['mov', 'pop'], ['mov', 'jmp'], ['mov', 'mov', 'mov'], ['push', 'mov', 'sub'], ['mov', 'mov', 'mov', 'mov'], ['pop'], ['mov', 'mov', 'add'], ['mov', 'mov', 'mov', 'sub'], ['mov', 'push'], ['pop', 'mov', 'mov'], ['add', 'mov'], ['push', 'lea'], ['push', 'mov', 'mov'], ['lea', 'push', 'call', 'add'], ['push', 'lea', 'push', 'lea'], ['push', 'c

# opcode list load

In [ ]:
filePath = '/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/files/opcode database/opcodesList.txt'
with open(filePath, 'rb') as lf:
    opcodes = pickle.load(lf)

# opcodes.remove('nop')

for i in range(len(opcodes)):
    opcodes[i] = [opcodes[i]]

print(opcodes)

[['aaa'], ['aad'], ['aam'], ['aas'], ['adc'], ['add'], ['and'], ['call'], ['cbw'], ['clc'], ['cld'], ['cli'], ['cmc'], ['cmp'], ['cmpsb'], ['cmpsw'], ['cwd'], ['daa'], ['das'], ['dec'], ['div'], ['esc'], ['hlt'], ['idiv'], ['imul'], ['in'], ['inc'], ['int'], ['into'], ['iret'], ['jcc'], ['ja'], ['jae'], ['jb'], ['jbe'], ['jc'], ['je'], ['jg'], ['jge'], ['jl'], ['jle'], ['jnle'], ['jno'], ['jnp'], ['jns'], ['jnz'], ['jo'], ['jp'], ['jpe'], ['jpo'], ['js'], ['jz'], ['jcxz'], ['jmp'], ['lahf'], ['lds'], ['lea'], ['les'], ['lock'], ['lodsb'], ['lodsw'], ['loop'], ['loope'], ['loopne'], ['loopnz'], ['loopz'], ['loopx'], ['mov'], ['movsb'], ['movsw'], ['mul'], ['neg'], ['nop'], ['not'], ['or'], ['out'], ['pop'], ['popf'], ['push'], ['pushf'], ['rcl'], ['rcr'], ['repxx'], ['rep'], ['repe'], ['repne'], ['repnz'], ['repz'], ['ret'], ['retn'], ['retf'], ['rol'], ['ror'], ['sahf'], ['sal'], ['sar'], ['sbb'], ['scasb'], ['scasw'], ['shl'], ['shr'], ['stc'], ['std'], ['sti'], ['stosb'], ['stosw'], 

## vocab load

In [ ]:
with open('/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/BPE 알고리즘2/2000vocab.p', 'rb') as file:
    vocab = pickle.load(file)

vocab = vocab + opcodes

print(vocab)

[['add', 'add'], ['mov', 'mov'], ['add', 'add', 'add', 'add'], ['push', 'push'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['push', 'call'], ['push', 'mov'], ['dec', 'mov'], ['mov', 'mov', 'mov', 'mov'], ['pop', 'pop'], ['mov', 'cmp'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['test', 'je'], ['mov', 'mov', 'mov'], ['mov', 'jmp'], ['cmp', 'je'], ['mov', 'add'], ['nop', 'nop'], ['pop', 'ret'], ['push', 'push', 'call'], ['push', 'lea'], ['mov', 'xor'], ['mov', 'call'], ['push', 'call', 'add'], ['inc', 'add'], ['dec', 'mov', 'dec', 'mov'], ['dec', 'lea'], ['pop', 'mov'], ['add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add', 'add'], ['mov', 'inc'], ['mov', 'lea'], ['push', 'push', 'push', 'push'], ['push', 'mov', 'call'], ['mov', 'push'], ['a

In [ ]:
vocab_number = [len(vocab) - i for i in range(len(vocab))]

print(vocab_number)

[2112, 2111, 2110, 2109, 2108, 2107, 2106, 2105, 2104, 2103, 2102, 2101, 2100, 2099, 2098, 2097, 2096, 2095, 2094, 2093, 2092, 2091, 2090, 2089, 2088, 2087, 2086, 2085, 2084, 2083, 2082, 2081, 2080, 2079, 2078, 2077, 2076, 2075, 2074, 2073, 2072, 2071, 2070, 2069, 2068, 2067, 2066, 2065, 2064, 2063, 2062, 2061, 2060, 2059, 2058, 2057, 2056, 2055, 2054, 2053, 2052, 2051, 2050, 2049, 2048, 2047, 2046, 2045, 2044, 2043, 2042, 2041, 2040, 2039, 2038, 2037, 2036, 2035, 2034, 2033, 2032, 2031, 2030, 2029, 2028, 2027, 2026, 2025, 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959, 1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948, 1947, 194

# opcode list 숫자로 변환

In [ ]:
inputList = []

for i in tqdm.notebook.tqdm(range(len(opcodeSequenceList))):
    tempList = []
    for j in range(len(opcodeSequenceList[i])):
        tempList.append(vocab_number[vocab.index(opcodeSequenceList[i][j])])
    inputList.append(tempList)

print(inputList[0])

  0%|          | 0/10854 [00:00<?, ?it/s]

[1943, 1867, 393, 1527, 1099, 59, 2107, 1285, 1999, 1786, 66, 2088, 2093, 1313, 1942, 1867, 2062, 1784, 1946, 2099, 2106, 13, 2043, 2098, 2099, 1943, 2104, 36, 2055, 1800, 2079, 1735, 2064, 2092, 2030, 640, 1551, 2089, 1944, 1062, 2092, 2030, 640, 1191, 2062, 1944, 2099, 88, 183, 1651, 2096, 1521, 1800, 2040, 1366, 1971, 2055, 1390, 105, 1556, 1681, 1725, 1313, 1942, 2104, 1762, 2024, 1165, 2024, 1986, 1165, 2018, 1942, 1867, 2099, 2063, 936, 1840, 2076, 1654, 650, 2082, 2093, 56, 700, 99, 1516, 1935, 2089, 1743, 1934, 59, 1888, 229, 1101, 1577, 59, 142, 108, 2011, 20, 1920, 730, 215, 1970, 12, 86, 1078, 2088, 108, 2011, 1746, 1466, 863, 2112, 1818, 103, 108, 2011, 1586, 1638, 195, 508, 2045, 12, 93, 1568, 204, 1, 1746, 1912, 1746, 1864, 863, 2112, 1818, 40, 1178, 1178, 38, 1638, 221, 2061, 1731, 935, 1974, 961, 2097, 1896, 1950, 1688, 1974, 1896, 1852, 1157, 106, 2042, 2094, 1928, 1, 2048, 2067, 41, 1779, 6, 2083, 2105, 13, 2064, 1685, 2077, 431, 29, 2100, 2083, 2105, 2094, 1996, 1888

In [39]:
inputList = np.array(inputList)
classList = [0 for i in range(len(opcodeSequenceList))]

for i in range(8639):
    classList[i] = 1

classList = np.array(classList)

X_train, X_test, Y_train, Y_test = train_test_split(inputList, classList, random_state = 42, test_size=0.2)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(8683,) (8683,)
(2171,) (2171,)


In [53]:
model= Sequential()
model.add(Embedding(len(vocab), 3000))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding = 'valid', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(48, activation='relu'))
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 3000)        6336000   
                                                                 
 dropout_6 (Dropout)         (None, None, 3000)        0         
                                                                 
 conv1d_6 (Conv1D)           (None, None, 64)          960064    
                                                                 
 max_pooling1d_6 (MaxPooling  (None, None, 64)         0         
 1D)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 55)                26400     
                                                                 
 dense_12 (Dense)            (None, 48)                2688      
                                                      

In [54]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)
keras.initializers.he_uniform(seed)

In [55]:
# 모델을 저장할 경로를 설정해주시면 됩니다.
filename = 'BPESequenceBasedLSTMCNNMalwareDetectionModel.h5'
checkpoint = ModelCheckpoint(filename,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max'
                            )

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [56]:
x_train = sequence.pad_sequences(X_train, maxlen=3000)
x_test = sequence.pad_sequences(X_test, maxlen=3000)

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

In [ ]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test), callbacks=[checkpoint])
model.load_weights(filename)

In [58]:
model.load_weights(filename)

In [59]:
y_pred = model.predict(x_test)

Y_pred = np.argmax(y_pred,axis=1)

print(f'accuracy: {accuracy_score(Y_test, Y_pred)}')

accuracy: 0.9907876554583142
